In [1]:
import ee
import geemap
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as mcolors

In [2]:
# Initialize the project
ee.Authenticate()
ee.Initialize(project='shrubwise-dc')

In [3]:
fbfm40 = ee.ImageCollection("projects/sat-io/open-datasets/landfire/FUEL/FBFM40")
print(fbfm40.getInfo())

{'type': 'ImageCollection', 'bands': [], 'version': 1742160507671454, 'id': 'projects/sat-io/open-datasets/landfire/FUEL/FBFM40', 'features': [{'type': 'Image', 'bands': [{'id': 'F40', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [125270, 65974], 'crs': 'EPSG:3338', 'crs_transform': [30, 0, -2175045.0000000033, 0, -30, 2383724.9999999995]}], 'version': 1742160507411706, 'id': 'projects/sat-io/open-datasets/landfire/FUEL/FBFM40/F40_LA_240', 'properties': {'keywords': 'landfire, vegetation, fuels, disturbance, fire', 'references': 'https://landfire.gov/', 'project': 'LANDFIRE', 'institution': 'USGS', 'system:footprint': {'type': 'LinearRing', 'coordinates': [[172.2480478944905, 53.02311792076253], [175.5077446849606, 49.057317252748305], [-178.55701174024898, 50.775466545787616], [-172.18174675349724, 52.12022000621032], [-158.46131956967503, 53.56498448743871], [-144.35443296461304, 53.232920100768425], [-131.06150741152382, 51.16261

In [4]:
print("Total images:", fbfm40.size().getInfo())

Total images: 4


In [5]:
# Reading Labels
df = pd.read_csv("./LF2024_FBFM40_250_CONUS/CSV_Data/LF24_F40_250.csv")

labels_by_value = {int(r["VALUE"]): str(r["FBFM40"]) for _, r in df.iterrows()}
classes = sorted(labels_by_value.keys())
n = len(classes)

# Generate distinct colors (categorical colormap)
cmap = cm.get_cmap("tab20", n)   
palette = [mcolors.rgb2hex(cmap(i)) for i in range(n)]

vis_params = {
    "min": 0,
    "max": n - 1,
    "palette": palette
}

legend_dict = {
    f"{classes[i]} - {labels_by_value[classes[i]]}": palette[i]
    for i in range(n)
}

C:\Users\Pedro\AppData\Local\Temp\ipykernel_22732\2900019165.py:9: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap("tab20", n)


In [6]:
conus = fbfm40.filter(ee.Filter.eq("system:index", "F40_LC_240")).first()
remapped = conus.select("F40").remap(classes, list(range(n))).rename("FBFM40_idx")

# Visualize
Map = geemap.Map(center=[37.5, -120], zoom=6)
Map.addLayer(remapped, vis_params, "FBFM40")

# Add legend
Map.add_legend(title="FBFM40 Fuel Models", legend_dict=legend_dict)
Map

Map(center=[37.5, -120], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', …